In [9]:
from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import warnings
import tensorflow as tf
import seaborn as sns
import sklearn
import random
import math
import time


from lime.lime_tabular import RecurrentTabularExplainer
from tqdm import tqdm
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, r2_score 
from sklearn.model_selection import GroupKFold
from sklearn import preprocessing
from keras import backend as K
from sklearn.preprocessing import MinMaxScaler , StandardScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation, GRU
from scipy import optimize
from tensorflow.keras import optimizers


from sp_modif.model_function import *
from sp_modif.methods import *
from sp_modif.data_prep import *
from sp_modif.evaluator import *
from sp_modif.SHAP import *
from sp_modif.L2X import *
from methods import *

%matplotlib inline
warnings.filterwarnings('ignore')

SEED = 0
def set_seed(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(SEED)
    random.seed(SEED)
    np.random.seed(SEED)
    tf.random.set_seed(SEED)

# Appeler la fonction pour fixer le seed
set_seed(SEED)

In [10]:
# Load data
# Load data and preprocess
train, test, y_test = prepare_data('FD004.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

remaining_sensors = ['T24','T30','T50', 'P15', 'P30','Nf','Nc', 'epr','Ps30','phi',
                    'NRf','NRc','BPR', 'farB','htBleed','W31','W32']
drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

rul_piecewise = 120
train['RUL'].clip(upper=rul_piecewise, inplace=True)

(61249, 27) (41214, 26) (248, 1)


In [17]:
def model_lstm_1layer(input_shape, nodes_per_layer, dropout, activation):
    
    cb = keras.callbacks.EarlyStopping(monitor='loss', patience=4)
    model = Sequential()
    model.add(LSTM(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(Dense(256))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',
                  optimizer=Adam(learning_rate=0.001))
    # model.save_weights(weights_file)

    return model

In [18]:
%%time
results_all = pd.DataFrame()
for SEED in range(5):  
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    # 0.20	[64]	0.3	tanh	32	25
    
    # parameter's sample
    # weights_file = "weights_file_lstm_optimalmodel_all.h5"
    alpha = 0.3
    sequence_length = 40
    epochs = 20
    nodes_per_layer = [64]
    dropout = 0.2
    activation = 'tanh'
    batch_size = 32
    remaining_sensors = remaining_sensors
    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout, activation)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
            
    # Model fitting
    with tf.device('/device:GPU:0'):
        start_time = time.time()
        model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))
        weights_file = model.get_weights()
        model.set_weights(weights_file)  # reset optimizer and node weights before every training iteration
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        end_time = time.time()
        training_time = end_time - start_time
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length,
         'time':training_time}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results_all = pd.concat([results_all, pd.DataFrame(d, index=[0])], ignore_index=True)
    results_all.to_csv('results/all/fd004.csv')

(51538, 40, 17) (51538, 1) (248, 40, 17)
Epoch 1/20
1611/1611 [==============================] - 13s 8ms/step - loss: 890.6100 - val_loss: 475.7899
Epoch 2/20
1611/1611 [==============================] - 12s 7ms/step - loss: 326.5423 - val_loss: 272.6232
Epoch 3/20
1611/1611 [==============================] - 12s 8ms/step - loss: 283.3440 - val_loss: 248.6553
Epoch 4/20
1611/1611 [==============================] - 12s 8ms/step - loss: 263.2531 - val_loss: 267.4348
Epoch 5/20
1611/1611 [==============================] - 12s 7ms/step - loss: 246.8677 - val_loss: 244.3410
Epoch 6/20
1611/1611 [==============================] - 12s 7ms/step - loss: 229.2129 - val_loss: 284.2966
Epoch 7/20
1611/1611 [==============================] - 13s 8ms/step - loss: 216.3931 - val_loss: 273.7994
Epoch 8/20
1611/1611 [==============================] - 12s 7ms/step - loss: 207.4074 - val_loss: 224.3848
Epoch 9/20
1611/1611 [==============================] - 12s 8ms/step - loss: 199.3400 - val_loss: 221.5

In [20]:
results_all

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size,TW
0,13.978763,0.0,942.298756,0.0,195.405823,0.0,[64],0.2,tanh,32,40
1,13.408881,0.0,893.612929,0.0,179.798080,0.0,[64],0.2,tanh,32,40
2,14.474265,0.0,1015.154789,0.0,209.504333,0.0,[64],0.2,tanh,32,40
3,14.056982,0.0,756.770995,0.0,197.598755,0.0,[64],0.2,tanh,32,40
4,13.087103,0.0,766.305731,0.0,171.272263,0.0,[64],0.2,tanh,32,40


### FD002 <a class="anchor" id="fd002"></a>

In [22]:
# Load data
# Load data and preprocess
train, test, y_test = prepare_data('FD002.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

remaining_sensors = ['T24','T30','T50', 'P15', 'P30','Nf','Nc', 'epr','Ps30','phi',
                    'NRf','NRc','BPR','htBleed','W31','W32']
drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

rul_piecewise = 125
train['RUL'].clip(upper=rul_piecewise, inplace=True)

(53759, 27) (33991, 26) (259, 1)


In [23]:
%%time
results_all002 = pd.DataFrame()
for SEED in range(5):  
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    # parameter's sample
    # weights_file = "weights_file_lstm_optimalmodel_all.h5"
    alpha = 0.2
    sequence_length = 40
    epochs = 20
    nodes_per_layer = [32]
    dropout = 0.1
    activation = 'tanh'
    batch_size = 128
    remaining_sensors = remaining_sensors
    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout, activation)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
            
    # Model fitting
    with tf.device('/device:GPU:0'):
        start_time = time.time()
        model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))
        weights_file = model.get_weights()
        model.set_weights(weights_file)  # reset optimizer and node weights before every training iteration
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        end_time = time.time()
        training_time = end_time - start_time
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length,
         'time':training_time}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results_all002 = pd.concat([results_all002, pd.DataFrame(d, index=[0])], ignore_index=True)
    results_all002.to_csv('results/all/fd002.csv')

(43619, 40, 16) (43619, 1) (259, 40, 16)
Epoch 1/20
341/341 [==============================] - 5s 11ms/step - loss: 2299.9543 - val_loss: 1355.9552
Epoch 2/20
341/341 [==============================] - 3s 10ms/step - loss: 445.4123 - val_loss: 230.4746
Epoch 3/20
341/341 [==============================] - 3s 10ms/step - loss: 272.6965 - val_loss: 213.4318
Epoch 4/20
341/341 [==============================] - 3s 10ms/step - loss: 250.4840 - val_loss: 199.3014
Epoch 5/20
341/341 [==============================] - 3s 10ms/step - loss: 241.4520 - val_loss: 191.6943
Epoch 6/20
341/341 [==============================] - 3s 9ms/step - loss: 235.6269 - val_loss: 194.9564
Epoch 7/20
341/341 [==============================] - 3s 8ms/step - loss: 231.8452 - val_loss: 183.0522
Epoch 8/20
341/341 [==============================] - 3s 8ms/step - loss: 229.6745 - val_loss: 196.7726
Epoch 9/20
341/341 [==============================] - 3s 8ms/step - loss: 222.4664 - val_loss: 195.1641
Epoch 10/20
341/

In [24]:
results_all002

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size,TW
0,13.375069,0.0,880.479340,0.0,178.892471,0.0,[32],0.1,tanh,128,40
1,13.638212,0.0,1063.441856,0.0,186.000839,0.0,[32],0.1,tanh,128,40
2,13.146090,0.0,883.644756,0.0,172.819672,0.0,[32],0.1,tanh,128,40
3,13.592679,0.0,916.008385,0.0,184.760925,0.0,[32],0.1,tanh,128,40
4,14.689135,0.0,831.363601,0.0,215.770660,0.0,[32],0.1,tanh,128,40


### FD003 <a class="anchor" id="fd003"></a>

In [25]:
# Load data
# Load data and preprocess
train, test, y_test = prepare_data('FD002.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

remaining_sensors = ['T24','T30','T50', 'P15', 'P30','Nf','Nc', 'epr','Ps30','phi',
                    'NRf','NRc','BPR','htBleed','W31','W32']
drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

rul_piecewise = 125
train['RUL'].clip(upper=rul_piecewise, inplace=True)

(53759, 27) (33991, 26) (259, 1)


In [26]:
%%time
results_all003 = pd.DataFrame()
for SEED in range(5):  
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    # parameter's sample
    # weights_file = "weights_file_lstm_optimalmodel_all.h5"
    alpha = 0.1
    sequence_length = 35
    epochs = 20
    nodes_per_layer = [64]
    dropout = 0.2
    activation = 'tanh'
    batch_size = 32
    remaining_sensors = remaining_sensors
    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout,
                             activation)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
            
    # Model fitting
    with tf.device('/device:GPU:0'):
        start_time = time.time()
        model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))
        weights_file = model.get_weights()
        model.set_weights(weights_file)  # reset optimizer and node weights before every training iteration
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        end_time = time.time()
        training_time = end_time - start_time
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length,
         'time':training_time}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results_all003 = pd.concat([results_all003, pd.DataFrame(d, index=[0])], ignore_index=True)
    results_all003.to_csv('results/all/fd003.csv')

(44919, 35, 16) (44919, 1) (259, 35, 16)
Epoch 1/20
1404/1404 [==============================] - 9s 5ms/step - loss: 875.2603 - val_loss: 262.0320
Epoch 2/20
1404/1404 [==============================] - 7s 5ms/step - loss: 267.9233 - val_loss: 222.1751
Epoch 3/20
1404/1404 [==============================] - 7s 5ms/step - loss: 244.3946 - val_loss: 201.0150
Epoch 4/20
1404/1404 [==============================] - 7s 5ms/step - loss: 230.2976 - val_loss: 183.0546
Epoch 5/20
1404/1404 [==============================] - 7s 5ms/step - loss: 219.2994 - val_loss: 172.6591
Epoch 6/20
1404/1404 [==============================] - 6s 5ms/step - loss: 210.1452 - val_loss: 166.0017
Epoch 7/20
1404/1404 [==============================] - 6s 4ms/step - loss: 204.7170 - val_loss: 171.0595
Epoch 8/20
1404/1404 [==============================] - 6s 5ms/step - loss: 198.9625 - val_loss: 175.7807
Epoch 9/20
1404/1404 [==============================] - 6s 5ms/step - loss: 193.3466 - val_loss: 175.8752
Epoch

In [27]:
results_all003

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size,TW
0,12.443282,0.0,669.450046,0.0,154.835281,0.0,[64],0.2,tanh,32,35
1,12.380057,0.0,648.540484,0.0,153.265808,0.0,[64],0.2,tanh,32,35
2,12.791228,0.0,744.168177,0.0,163.615524,0.0,[64],0.2,tanh,32,35
3,13.043703,0.0,931.461167,0.0,170.138184,0.0,[64],0.2,tanh,32,35
4,12.710529,0.0,611.947600,0.0,161.557556,0.0,[64],0.2,tanh,32,35


### FD001

In [29]:
# Load data
# Load data and preprocess
train, test, y_test = prepare_data('FD001.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']
remaining_sensors = ['T24','T30','T50', 'P15', 'P30','Nf','Nc','Ps30','phi',
                    'NRf','NRc','BPR','htBleed','W31','W32']
drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

rul_piecewise = 125
train['RUL'].clip(upper=rul_piecewise, inplace=True)

(20631, 27) (13096, 26) (100, 1)


In [30]:
%%time
results_all001 = pd.DataFrame()
for SEED in range(5):  
    set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    # parameter's sample
    # weights_file = "weights_file_lstm_optimalmodel_all.h5"
    alpha = 0.1
    sequence_length = 30
    epochs = 20
    nodes_per_layer = [128]
    dropout = 0.2
    activation = 'tanh'
    batch_size = 64
    remaining_sensors = remaining_sensors
    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout,
                             activation)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
            
    # Model fitting
    with tf.device('/device:GPU:0'):
        start_time = time.time()
        model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))
        weights_file = model.get_weights()
        model.set_weights(weights_file)  # reset optimizer and node weights before every training iteration
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        end_time = time.time()
        training_time = end_time - start_time
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length,
         'time':training_time}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results_all001 = pd.concat([results_all001, pd.DataFrame(d, index=[0])], ignore_index=True)
    results_all001.to_csv('results/all/fd001.csv')

(17731, 30, 15) (17731, 1) (100, 30, 15)
Epoch 1/20
278/278 [==============================] - 4s 11ms/step - loss: 1945.0713 - val_loss: 1142.2662
Epoch 2/20
278/278 [==============================] - 3s 9ms/step - loss: 594.9089 - val_loss: 336.2231
Epoch 3/20
278/278 [==============================] - 3s 10ms/step - loss: 349.9518 - val_loss: 213.0034
Epoch 4/20
278/278 [==============================] - 3s 11ms/step - loss: 243.1687 - val_loss: 380.7289
Epoch 5/20
278/278 [==============================] - 3s 11ms/step - loss: 222.0929 - val_loss: 180.0421
Epoch 6/20
278/278 [==============================] - 3s 11ms/step - loss: 203.6436 - val_loss: 266.0940
Epoch 7/20
278/278 [==============================] - 3s 11ms/step - loss: 200.1159 - val_loss: 201.9042
Epoch 8/20
278/278 [==============================] - 3s 10ms/step - loss: 189.3215 - val_loss: 217.6176
Epoch 9/20
278/278 [==============================] - 3s 11ms/step - loss: 187.8194 - val_loss: 170.0289
Epoch 10/20
2